# Exploring Shape of AMM Curves

In Module 3 of the TE Fundamentals course, there is a deep dive into the properties of the Uniswap v2 constant-product market maker.

In this session of TE Exercises, we explore another example of an AMM curve, as well as general theoretical properties that AMM curves should satisfy.

## StableSwap

The *StableSwap* invariant proposed by Michael Egorov is an AMM mechanism designed to work especially well for stablecoins.

**Assumptions:**
* $x$ is the amount of Token $X$
* $y$ is the amount of Token $Y$
* $A$ is a parameter set by the pool which controls the shape of the curve.
* $D$ is a parameter initially set to which always satisfies $$4A(x+y) + D = 4AD + \displaystyle\frac{D^3}{4xy}$$
* The curve has no fees

Then the mechanics of the curve are governed by this equation:

$$4A(x+y) + D = 4AD + \displaystyle\frac{D^3}{4xy}$$

Where $A$ is a parameter according to the pool designer's choice.

**References**:
- [Original StableSwap Paper](https://web.archive.org/web/20211018201638/https://curve.fi/files/stableswap-paper.pdf)
- [Post on Understanding the StableSwap Curve](https://miguelmota.com/blog/understanding-stableswap-curve/)

## Basic Exercises

**Exercise 1:**  Use desmos.com to graph the function with $A$ as a slider.

- (a) As $A$ grows larger, how does the curve change?
ANSWER: The curve flattens out and moves from a constant product curve to a constant sum curve

<details>
    <summary><b> Answer to Exercise 1(a): </b>  </summary>
As $A$ increases, the curve becomes more linear.
</details>

- (b)What does the curve look like when $A = 0$
ANSWER: It looks just like a constant product AMM curve

<details>
    <summary><b> Answer to Exercise 1(b): </b> </summary> When $A = 0$, the curve looks like a constant-product market maker $xy = k$.
</details>


**Exercise 2:** In this exercise we look in greater depth at the equation for StableSwap, and the $D$ invariant in particular.

- (a) Rewrite the equation so it has $0$ on one side. Call the expression on the other side (involving $A$, $D$, $x$, and $y$) $F_A(D,x,y)$. This is the invariant function.
<details>
    <summary><b> Answer to Exercise 2(a): </b>  </summary>
You get $$4A(x+y) + D - 4AD - \displaystyle\frac{D^3}{4xy} = 0.$$
</details>

- (b) Simplify the equation when $A = 0$. What does this look like?
<details>
    <summary><b> Answer to Exercise 2(b): </b>  </summary>
You get $$D -  \displaystyle\frac{D^3}{4xy} = 0.$$ This can be simplified back into $$4Dxy - \displaystyle\frac{D^3} = 0$$, which simplifies to $$xy = \frac{D^2}{4}.$$ In other words, we have a constant-product market maker again.
</details>
- (c) Set up (but do not attempt to solve) an equation that will find $D$ in terms of $A$, $x$, and $y$.
    <details>
    <summary><b> Answer to Exercise 2(c): </b>  </summary>
Setting up the equation involves isolating all $D$ terms on one side: $$4A(x+y)  =  -D + 4AD + \displaystyle\frac{D^3}{4xy}.$$ We can rewrite this as $$$A(x+y) = (1-4A)D + \displaystyle\frac{D^3}{4xy}.$$
</details>
- (d) If you feel like having fun, solve for $D$ symbolically in terms of $A$, $x$, and $y$.
    <details>
    <summary><b> Answer to Exercise 2(d): </b>  </summary>
Starting with $4A(x+y) = (4A - 1)D + \displaystyle\frac{D^3}{4xy}$, which we rewrite as $\displaystyle\frac{1}{4xy}D^3 + (1-4A)D - 4a(x+y) = 0$. This is a *depressed cubic*, and we need to use the [Cubic Formula](https://math.vanderbilt.edu/schectex/courses/cubic/) to solve this equation with $a = \frac{1}{4xy}$, $b = 0$, $c = 4A - 1$ and $d = 4A(x+y)$.
</details>
- (e) In practice, how is this equation for $D$ solved?
    <details>
    <summary><b> Answer to Exercise 2(e): </b>  </summary>
The cubic formula requires a square root, which unfortunately is hard to implement in languages like Solidity. Instead, [Newton's Method](https://calcworkshop.com/derivatives/newtons-method/) is used to solve the equation.
</details>

**Exercise 3:** Write a function ```invariant_function``` that takes as input $A$, $D$, $x$, and $y$, and calculates the value of the invariant function.


```python
def invariant_function(A,D,x,y):
   val = 1/(4*x*y)*D**3 + (4*A - 1)*D - 4*A*(x+y)
   return val

print(invariant_function(2,3,100,100));
```


In [39]:
def invariant_function(A,D,x,y):
   val = 1/(4*x*y)*D**3 + (4*A - 1)*D - 4*A*(x+y)
   return val

print(invariant_function(0,0.1,100,100));

-0.099999975


**Exercise 4:** What are errors that you would need to check for in calculating ```invariant_function```?
ANSWER: That the parameters are real numbers. All must be greater than zero, except A, which is the only one that can be zero
<details>
<summary><b> Answer to Exercise 4: </b>  </summary>

<ul>
These are the conditions that should be true on the variables.
<li> $x > 0$ </li>
<li> $y > 0$ </li>
<li> $A \geq 0$ </li>
<li> $D > 0$ </li>
</ul>

</details>

**Exercise 5:** Write a function ```is_valid_state``` that takes as input $A$, $D$, $x$, and $y$ and determines if the state could exist in a valid implementation of the StableSwap AMM.

```python
def is_valid_state(A,D,x,y):
  value = invariant_function(A,D,x,y) #evaluate the function
  valid = (value == 0) #see if it returns 0
  return valid
```

**Note:** In practice, the above function would need to be modified to check whether the value was appropriately small. Due to numerical roundoff issues, it is possible that a valid state might not be exactly equal to 0. The line

```valid = (value == 0) ```

could be replaced to use the [math.isclose](https://docs.python.org/3/library/math.html#math.isclose) function. However, this has the drawback that a slightly invalid state might be accepted. We would need to do some numerical analysis and experimentation to determine how close we would accept as being equal.

In [42]:
def is_valid_state(A,D,x,y):
  value = invariant_function(A,D,x,y) #evaluate the function
  valid = (value == 0) #see if it returns 0
  return valid

print(is_valid_state(10,20,20,20));

False


**Exercise 6:** Write a function ```is_valid_trade``` that takes six inputs
- the parameter $A$
- the value of $D$
- the initial value $x_{\text{old}}$
- the initial value $y_{\text{old}}$
- the value $x_{\text{new}}$ achieved after trading
- the value $y_{\text{new}}$ achieved after trading.

 ```python
def is_valid_trade(A,D, x_old, y_old,
                   x_new, y_new):
    is_valid_state_old = is_valid_state(A,D,x_old,y_old)
    is_valid_state_new = is_valid_state(A,D,x_new, y_new)
    is_valid = is_valid_state_old and is_valid_state_new #logical check to see if both are valid states
    return is_valid

 ```


In [44]:
def is_valid_trade(A,D, x_old, y_old,
                 x_new, y_new):
  is_valid_state_old = is_valid_state(A,D,x_old,y_old)
  is_valid_state_new = is_valid_state(A,D,x_new, y_new)
  is_valid = is_valid_state_old and is_valid_state_new #logical check to see if both are valid states
  return is_valid

print(is_valid_trade(10,20,100,100,
                 500, 20))

False


## Advanced Exercises

**Definition:** A function of two variables is called *homogeneous of degree $n$* ($n > 0$) is a function $F(x,y)$ such that $F(rx,ry) = r^nF(x,y)$.

## Exercise 1

Show that a constant-product market maker invariant function is homogeneous of degree 2.

<details>
<summary> Solution to Advanced Exercise 1: </summary>
If $F(x,y) = xy$, then $F(rx,ry) = (rx)(ry) = r^2xy = r^2F(x,y)$. So by definition, this function is homogenous of degree 2.
</details>

## Exercise 2

Prove that a two-variable homogeneous function $G$ will always have $G(0,0) = 0$.

<details>
<summary> Solution to Advanced Exercise 2
</summary>
By definition of homogeneous function, for any $r$ we have $$G(0,0) = G(r \cdot 0, r \cdot 0) = r^2 G(0,0).$$ This condition can not hold for $r = 2$ unless $G(0,0) = 0$.
</details>

## Exercise 3

A constant-function market maker implicitly gives $y$ as a function of $x$. Let's call this implicit function $y = f(x)$. What properties should this function have, in terms of the first and second derivative?

<details>
<summary>
Solution to Advanced Exercise 3
</summary>
* **First Derivative:** It should be true that $\displaystyle\frac{dy}{dx} <  0$ for any value of $x$. **Reasoning:** As the amount of one token increases, the amount of the other token must necessarily decrease. So if $x$ increases, $y$ should decrease, and $\displaystyle\frac{dy}{dx} < 0$.
* **Second Derivative:** It should be true that $\displaystyle\frac{d^2y}{dx} > 0$ for any value of $x$. **Reasoning:** The spot price for Token Y is given $-\displaystyle\frac{dy}{dx}$. If $x$ increases, the spot price $-\displaystyle\frac{dy}{dx}$ should decrease (more Token X in the pool makes Token Y cheaper). This means $\frac{dy}{dx}$ increases, so $\displaystyle\frac{d^2y}{dx^2} > 0$.
</details>

## Exercise 4

Explain why the curve $x^2 + y^2 = k$ would be a poor choice for an AMM.

<details>
<summary>Solution to Advanced Exercise 4:
</summary>
Solving for $\frac{dy}{dx}$ (using implicit differentiation) gives $$\frac{dy}{dx} = \displaystyle\frac{x}{y}$$ For $x,y > 0$, this gives $\displaystyle\frac{dy}{dx} > 0$, which does not match the properties above.
</details>